

<h1>AnimeGANv2 Colab notebook</h1>
<a href="https://github.com/TachibanaYoshino/AnimeGANv2"><img  src="https://img.shields.io/badge/Open%20on-GitHub-blue?logo=github" alt="Open on GitHub" title="Open project on GitHub"></a>  

--- 
<br>
<img width="50px" align="left" src="https://img.icons8.com/plasticine/100/000000/general-warning-sign.png"/>

**Please check if you're using a GPU instance!**  
Click the “Runtime” dropdown menu.  Select “Change runtime type” and set it to "GPU"  
<br>

---
*Some utility cells can be found at the bottom.* [Scroll down](#scrollTo=r1vUP2QpQdf2)

## Prepare dependencies

In [ ]:
from IPython.display import clear_output
%tensorflow_version 1.x
!git clone https://github.com/TachibanaYoshino/AnimeGANv2
%cd AnimeGANv2/
clear_output()
print("Done preparing dependencies")

# Configuration


**Option** | **Value** | **Description**
---|---|---
STYLE | [ Paprika, Hayao, Shinkai ] | Which style to apply (Shinkai is not available yet)
TYPE | [ image, video ] | Set this to the type of input you're uploading
DOWNSCALE_VIDEO | [True, False] | If true, downscales the video to 720p for faster processing 
STORAGE | [local, drive, none] | **local**: download the processed video <br>**drive**: save it to Google Drive (creates AnimeGANv2 subfolder)




In [2]:
STYLE="Hayao" #@param ["Paprika", "Hayao", "Shinkai"]
TYPE="video" #@param ["image", "video" ]
DOWNSCALE_VIDEO=False #@param {type:"boolean"}
STORAGE="drive" #@param ["local", "drive", "none"]

#Mount Google Drive if required
if STORAGE is "drive":
  from google.colab import drive
  drive.mount('/content/drive')
  clear_output()
  print('Drive mounted and ', end='')
print('options set')

Drive mounted and options set


## Upload files
Upload one or more images/videos

In [ ]:
import os
from google.colab import files

!mkdir -p "/content/AnimeGANv2/{TYPE}/input/" 2>/dev/null
%cd "/content/AnimeGANv2/{TYPE}/input/"
uploaded = files.upload()
for fn in uploaded.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(
      name=fn, length=len(uploaded[fn])))

if TYPE is "video" and DOWNSCALE_VIDEO is True:
  !mkdir /content/AnimeGANv2/video/original/ 2>/dev/null
  !mv /content/AnimeGANv2/video/input/* "/content/AnimeGANv2/video/original"
  for file in os.listdir('/content/AnimeGANv2/video/original'):
    print("Downscaling video " + file)
    !ffmpeg -hide_banner -loglevel warning -y -r 24 -i "/content/AnimeGANv2/video/original/{file}" -filter:v scale="trunc(oh*a/2)*2:720"  -c:a copy "/content/AnimeGANv2/video/input/{file}"
print("Done uploading")

/content/AnimeGANv2/video/input


Saving VID_20200811_174641.mp4 to VID_20200811_174641.mp4
Saving VID_20200811_172917.mp4 to VID_20200811_172917.mp4


## Start processing
This step will take a while. Results are automatically saved to your selected storage

In [ ]:
%cd /content/AnimeGANv2/
!rm /content/AnimeGANv2/results.zip 2>/dev/null

if TYPE is "video":
  for file in os.listdir('/content/AnimeGANv2/video/input/'):
    !echo "checkpoint/generator_{STYLE}_weight > {file}"
    !python video2anime.py --video "/content/AnimeGANv2/video/input/{file}"  --checkpoint_dir "checkpoint/generator_{STYLE}_weight" --output "results/{STYLE}/output" 
  %cd "/content/AnimeGANv2/video/output/{STYLE}/"
  !zip -r /content/AnimeGANv2/results.zip *
elif TYPE is "image":
  !echo "checkpoint/generator_{STYLE}_weight > /content/AnimeGANv2/image/input"
  !python test.py --checkpoint_dir "checkpoint/generator_{STYLE}_weight" --test_dir "/content/AnimeGANv2/image/input" --style_name "{STYLE}/output"
  %cd "/content/AnimeGANv2/results/{STYLE}/output/"
  !zip -r /content/AnimeGANv2/results.zip *
clear_output()


#Saving
if STORAGE is "local":
  files.download('/content/AnimeGANv2/results.zip')
elif STORAGE is "drive":
  !mkdir -p "/content/drive/My Drive/AnimeGANv2/{STYLE}" 2>/dev/null
  !cp "/content/AnimeGANv2/results/{STYLE}/output/"* "/content/drive/My Drive/AnimeGANv2/{STYLE}/"
  print("File copied to Google Drive")


## Utility cells

In [ ]:
# Clear inputs
!rm -rf /content/AnimeGANv2/video/original/* /content/AnimeGANv2/video/input/* 
!rm -rf /content/AnimeGANv2/image/input/*

In [ ]:
# Clear outputs
!rm -rf /content/AnimeGANv2/video/output/* 
!rm -rf /content/AnimeGANv2/results
!rm /content/AnimeGANv2/results.zip 2>/dev/null